In [37]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#### Categorias do Documento  
Como explicado na documentação, os textos são divididos em algumas categorias. Para processar os dados, deve-se converter essas strings em números.  
É importante lembrar que antes de executar o programa deve-se descompactar o arquivo *rcv1rcv2aminigoutte.tar.gz* na pasta do projeto. Isso criará a pasta *rcv1rcv2aminigoutte*, que contém os arquivos necessários para o problema.

In [2]:
categories = {'C15': 4587, 'CCAT': 8745, 'E21': 9625, 'ECAT': 5656, 'GCAT': 8745, 'M11': 45845}

Esse método faz esse tratamento dos dados, substituindo as strings correspondentes as classes em seus respectivos números.

In [3]:
def filter_datas(file, target_file):
    with open(file, 'r') as document_read:
        with open(target_file, 'w') as document_write:
            for line in document_read:
                target = line.split(None, 1)[0]
                line_to_write = '%s %s' % (categories[target], line[len(target)+1:len(line)])
                document_write.write(line_to_write)
        
    return target_file

#### Carregando o dataset no formato SVMLight

In [9]:
file = filter_datas('rcv1rcv2aminigoutte/EN/Index_EN-EN', 'svml_en_en.txt')
X,  y = load_svmlight_file(file)

#### Aplicando a seleção das features

In [10]:
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)

In [15]:
print('Data matrix shape before feature selection: ' + str(X.shape))
print('Data matrix shape after feature selection: ' + str(X_new.shape))

Data matrix shape before feature selection: (18758, 21531)
Data matrix shape after feature selection: (18758, 2973)


#### Dividindo o dataset
O método *train_test_split* é bem útil para dividir os dados em partes que serão usados para o treino do modelo e os testes, que serão convenientes para avaliar o modelo. Os dados de treino estarão em *x_train* e *y_train*, e os dados de teste em *x_test* e *y_test*, sendo que 25% dos dados serão usados para os testes e os outros 75% para o treinamento.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

#### Encontrando um bom k
Um dos parâmetros mais importantes no algoritmo do *KNeighborsClassifier* é o *k*, que significa quantos "vizinhos" mais próximos serão considerados para classificar um dado de teste. Por exemplo, se *k = 3*, serão analizadas as três classes vizinhas mais próximos, sendo que aquela que for mais comum irá classificar esse dado.

In [23]:
k_scores = []

for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    knn_score = knn.score(X_test, y_test)
    print('Score for k = ' + str(k) + ': ' + str(knn_score))
    
    k_scores.append(knn_score)

Score for k = 1: 0.7652452025586354
Score for k = 2: 0.746908315565032
Score for k = 3: 0.715138592750533
Score for k = 4: 0.7100213219616205
Score for k = 5: 0.694456289978678
Score for k = 6: 0.7059701492537314
Score for k = 7: 0.6985074626865672
Score for k = 8: 0.7017057569296375
Score for k = 9: 0.6931769722814499
Score for k = 10: 0.6974413646055437


Para encontrar um bom *k*, foram feitos 10 testes, com *k* variando de 1 até 10. O modelo é testado a cada iteração e a acurácia é aferida. O maior resultado desses testes é um forte candidato para a escolha do *k*.

In [24]:
np_k_scores = np.asarray(k_scores)
best_k = np.argmax(np_k_scores) + 1
print('Best k to KNN Classifier: ' + str(best_k))

Best k to KNN Classifier: 1


### Avaliando o modelo
O modelo é criado utilizando o *k* que foi encontrado acima.  
Os dados de treino são usados para treinar o modelo através do método *fit*, enquanto os dados de teste são usados no método *predict* para testar o modelo.

In [25]:
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

#### Matriz de confusão

In [38]:
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

[[ 927    2  345    0    2]
 [   9  171  329    5    0]
 [  33   10 2224   17    6]
 [   1    8  141  158    0]
 [  10    1  182    0  109]]


#### Relatório de classificação

In [39]:
report = classification_report(y_test, y_pred)
print(report)

             precision    recall  f1-score   support

     4587.0       0.95      0.73      0.82      1276
     5656.0       0.89      0.33      0.48       514
     8745.0       0.69      0.97      0.81      2290
     9625.0       0.88      0.51      0.65       308
    45845.0       0.93      0.36      0.52       302

avg / total       0.81      0.77      0.75      4690

